# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [60]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [61]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [79]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(inplace=True)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(10, 20, kernel_size=5),
            nn.Dropout2d(p=0.5),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(inplace=True)
        )
        self.layer3 = nn.Sequential(
            nn.Linear(320, 50),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.5)
        )
        self.layer4 = nn.Sequential(
            nn.Linear(50, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(-1, 320)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

#optimizer = torch.optim.SGD(model.parameters(),lr=0.001)

Next, we can start to train and evaluate!

In [80]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        #sum_loss = 0.0
        # TODO:forward + backward + optimize
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()
        #sum_loss += loss.item()
        #print (sum_loss)


100%|██████████| 468/468 [00:19<00:00, 23.97it/s]


#### Q5:
Please print the training and testing accuracy.

In [81]:
# evaluate
# TODO:calculate the accuracy using traning and testing dataset
acc = 0
num = 0
for images, labels in tqdm(test_loader):
    out_put = model(images)
    num += 128
    predicted = torch.max(out_put, 1)[1]
    #print (out_put)
    #print (torch.max(out_put,1))
    for i in range(predicted.size()[0]):
        if predicted[i] == labels[i]:
            acc += 1
print ('accuracy on test dataset',float(acc)/num)

acc = 0
num = 0
for images, labels in tqdm(train_loader):
    out_put = model(images)
    num += 128
    predicted = torch.max(out_put, 1)[1]
    #print (out_put.size())
    #print (predicted.size())
    #print (torch.max(out_put,1))
    for i in range(predicted.size()[0]):
        if predicted[i] == labels[i]:
            acc += 1
print ('accuracy on train dataset',float(acc)/num)

100%|██████████| 78/78 [00:01<00:00, 41.27it/s]


accuracy on test dataset 0.9527243589743589


100%|██████████| 468/468 [00:10<00:00, 43.62it/s]


accuracy on train dataset 0.9463641826923077
